In [5]:
%env GRB_LICENSE_FILE=/home/artem/gurobi/gurobi.lic
import gurobipy as gp
from gurobipy import GRB
from sklearn.linear_model import LinearRegression
import numpy as np

env: GRB_LICENSE_FILE=/home/artem/gurobi/gurobi.lic


In [6]:
#!/usr/bin/env python3.7

# Copyright 2023, Gurobi Optimization, LLC

# This example formulates and solves the following simple MIP model:
#  maximize
#        x +   y + 2 z
#  subject to
#        x + 2 y + 3 z <= 4
#        x +   y       >= 1
#        x, y, z binary

np.random.seed(0)
values = np.random.randint(0, 100, size=(10)).tolist()
bundles = np.random.randint(0, 2, size=(10, 5)).tolist()
#print(bundles)
#num_items = len(bundles[0])
#n = len(bundles)

# bundles random 18 bundles of length 10
reg = LinearRegression().fit(bundles, values)

count = 10
bigM = 6000
while count < 11:
    count += 1
    reg = LinearRegression().fit(bundles, values)
    coef = reg.coef_
    intercept = reg.intercept_
    print(coef)
    print(intercept)
    x_vectors = []
    x_distances = []
    for bundle in range(len(bundles)):
        try:
            m = gp.Model("mip1")
            m.Params.NodefileStart = 0.5
            m.Params.Threads = 48
            x = m.addVars(len(bundles[0]), vtype=GRB.BINARY, name="x")
            b = m.addVars(len(bundles), vtype=GRB.BINARY, name="b")
            r = m.addVar(vtype=GRB.CONTINUOUS, name="r")
            m.setObjective(r, GRB.MAXIMIZE)

            y_prediction = sum(x[k]*coef[k] for k in range(M)) + intercept

            m.addConstr(values[bundle] - y_prediction <= r)
            m.addConstr(y_prediction - values[bundle] <= r)

            for j in range(len(bundles)):
                m.addConstr(values[j] - y_prediction + bigM * b[j] >= r)
                m.addConstr(y_prediction - values[j] + bigM * (1 - b[j]) >= r)
            
            m.optimize()
            #get x values from gurobi
            all_vars = m.getVars()
            #create a an x vector
            vector = [v.X for v in all_vars if v.VarName[0] == 'x']
            print(vector)
            #get objective value
            print(m.getObjective().getValue())
            print(vector)
            

        except gp.GurobiError as e:
            print('Error code ' + str(e.errno) + ': ' + str(e))
        


[  6.88679245  38.73584906  -2.88679245   3.8490566  -18.86792453]
35.58490566037738
Changed value of parameter NodefileStart to 0.5
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter Threads to 48
   Prev: 0  Min: 0  Max: 1024  Default: 0
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 24 physical cores, 48 logical processors, using up to 48 threads
Optimize a model with 22 rows, 16 columns and 152 nonzeros
Model fingerprint: 0x857eb057
Variable types: 1 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e-01, 6e+03]
Found heuristic solution: objective 1.5283019
Presolve removed 22 rows and 16 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 48 available processors)

Solution count 1: 1.5283 

Optimal solution fou

In [8]:
coef = [54.06573723806331, 30.3918607675506, 26.64865738108338, 21.73335332188267, 22.54657742815401, 42.04490131394058, 26.37345605087732, 34.36623309059114, 34.53891186784021, 0, 0, 28.64796142082264, 0, 66.30683013875412, 35.83803685967155, 18.26737904533803]
intercept = -136

try:
    m = gp.Model("mip1")
    r = m.addVar(vtype=GRB.CONTINUOUS, name="r")
    x = m.addVars(len(coef), vtype=GRB.BINARY, name="x")
    b = m.addVars(1, vtype=GRB.BINARY, name="b")
    m.setObjective(r, GRB.MAXIMIZE)
    m.addConstr(-sum(x[k]*coef[k] for k in range(len(coef))) - r <= -470)
    m.addConstr(-sum(x[k]*coef[k] for k in range(len(coef))) - r + b[0]*533 >= -443)
    m.addConstr(sum(x[k]*coef[k] for k in range(len(coef))) - r - b[0]*533 >= -90)
    m.optimize()
    m.feasRelaxS(0, True, False, True)
    print(m.getObjective().getValue())
    
except gp.GurobiError as e:

    print('Error code ' + str(e.errno) + ': ' + str(e))

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 24 physical cores, 48 logical processors, using up to 24 threads
Optimize a model with 3 rows, 18 columns and 44 nonzeros
Model fingerprint: 0x350a98db
Variable types: 1 continuous, 17 integer (17 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+01, 5e+02]
Presolve removed 0 rows and 4 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 48 available processors)

Solution count 0

Model is infeasible or unbounded
Best objective -, best bound -, gap -

Solve phase I feasrelax model to determine minimal relaxation

Optimize a model with 3 rows, 21 columns and 47 nonzeros
Model fingerprint: 0x092a050b
Variable types: 4 continuous, 17 integer (17 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range

AttributeError: Unable to retrieve attribute 'x'

In [1]:
 R0: - 54.06573723806331 x[1] - 30.39186076755067 x[2]
   - 26.64865738108338 x[3] - 21.73335332188267 x[4]
   - 22.54657742815401 x[5] - 42.04490131394058 x[6]
   - 26.37345605087732 x[7] - 34.36623309059114 x[8]
   - 34.53891186784021 x[9] - 28.64796142082264 x[12]
   - 66.30683013875412 x[14] - 35.83803685967155 x[15]
   - 18.26737904533803 x[16] - r <= -470.1842427435893
 R50: - 54.06573723806331 x[1] - 30.39186076755067 x[2]
   - 26.64865738108338 x[3] - 21.73335332188267 x[4]
   - 22.54657742815401 x[5] - 42.04490131394058 x[6]
   - 26.37345605087732 x[7] - 34.36623309059114 x[8]
   - 34.53891186784021 x[9] - 28.64796142082264 x[12]
   - 66.30683013875412 x[14] - 35.83803685967155 x[15]
   - 18.26737904533803 x[16] + 533.8717946205495 b[24] - r
   >= -443.7539147733867
 R51: 54.06573723806331 x[1] + 30.39186076755067 x[2]
   + 26.64865738108338 x[3] + 21.73335332188267 x[4]
   + 22.54657742815401 x[5] + 42.04490131394058 x[6]
   + 26.37345605087732 x[7] + 34.36623309059114 x[8]
   + 34.53891186784021 x[9] + 28.64796142082264 x[12]
   + 66.30683013875412 x[14] + 35.83803685967155 x[15]
   + 18.26737904533803 x[16] - 533.8717946205495 b[24] - r
   >= -90.11787984716284

SyntaxError: invalid syntax (<ipython-input-1-6fb19ec10ca5>, line 1)

In [ ]:
# try:
#     # Create a new model
#     m = gp.Model("mip1")

#     # Create variables
#     x = m.addVar(vtype=GRB.BINARY, name="x")
#     y = m.addVar(vtype=GRB.BINARY, name="y")
#     z = m.addVar(vtype=GRB.BINARY, name="z")

#     # Set objective
#     m.setObjective(x + y + 2 * z, GRB.MAXIMIZE)

#     # Add constraint: x + 2 y + 3 z <= 4
#     m.addConstr(x + 2 * y + 3 * z <= 4, "c0")

#     # Add constraint: x + y >= 1
#     m.addConstr(x + y >= 1, "c1")

#     # Optimize model
#     m.optimize()

#     for v in m.getVars():
#         print('%s %g' % (v.VarName, v.X))

#     print('Obj: %g' % m.ObjVal)

# except gp.GurobiError as e:
#     print('Error code ' + str(e.errno) + ': ' + str(e))

# except AttributeError:
#     print('Encountered an attribute error')